In [7]:
# Librerias necesarias
import pandas as pd
import numpy as np
import json
import ast
import gzip

In [8]:
# Rutas de los datos:
ruta_games='Datasets originales/steam_games.json.gz'
ruta_reviews='Datasets originales/user_reviews.json.gz'
ruta_items='Datasets originales/users_items.json.gz'

## Games:
games_orig = []
with gzip.open(ruta_games, 'rt', encoding='utf-8') as archivo:
    lineas = archivo.readlines()
    for linea in lineas:
        games_orig.append(linea) 

# Quitamos el '\n' al final de cada linea:
for i in range(0,len(games_orig)):
    games_orig[i]= games_orig[i].replace("\n","")
# Transformo cada una de las cadenas a diccionario de Python:
for i in range(0,len(games_orig)):
    games_orig[i]= json.loads(games_orig[i])
    
# Transformamos a DataFrame:
games_orig = pd.DataFrame(games_orig)
games_orig.shape

(120445, 13)

In [9]:
## Reviews:
reviews = []
with gzip.open(ruta_reviews, 'rb') as archivo:
    for linea in archivo:
        reviews.append(ast.literal_eval(linea.decode('utf-8')))
reviews_orig = pd.DataFrame(reviews)
archivo.close()
reviews_orig.shape

(25799, 3)

In [5]:
## Items:
items = []
with gzip.open(ruta_items, 'rt', encoding="utf-8") as archivo:
    lineas = archivo.readlines()
for linea in lineas:
    items.append(ast.literal_eval(linea))
    
# Transformamos a DataFrame:
items_orig = pd.DataFrame(items)
items_orig.shape

(88310, 5)

In [ ]:
games = games_orig.copy()
reviews = reviews_orig.copy()
items = items_orig.copy()